In [1]:
import string
import re
from os import listdir
from nltk.corpus import stopwords

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import plot_model
import numpy as np
import nltk

2023-12-21 23:49:29.692872: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 23:49:30.054705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 23:49:30.054817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 23:49:30.123953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 23:49:30.265398: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 23:49:30.266855: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/suhas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

In [4]:
text=load_doc('review_polarity/txt_sentoken/pos/cv026_29325.txt')
text

'for those of us who weren\'t yet born when the 1960\'s rock \'n\' rolled around , monterey pop affords an affectionate glimpse of the music that influenced our parents to be hippies . \nfrom otis redding to jimi hendrix , janis joplin to the mamas and the papas , and jefferson airplane to the who , this documentary is jam-packed with contagious energy . \nbut i give fair warning that i will reveal the ending , which does not do the rest of the film the justice it deserves . \nshot in 1969 at an outdoor concert that precluded woodstock , the film defies the stereotype of the general population at the time . \nsure , some have painted their faces and smoke joints , but d . a . \npennebaker ( the war room , moon over broadway ) surprisingly chooses to show a broad spectrum of the audience . \nno matter who is watching , it all comes back to the talented musicians that stir your soul . \nthe excitement starts before the music even begins . \na young girl is cleaning thousands of seats and

In [5]:
# turn a doc into clean tokens
def clean_doc(doc):
  # split into tokens by white space
  tokens = doc.split()
  # prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  # remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
  # remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
  # filter out stop words
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
  # filter out short tokens
  tokens = [word for word in tokens if len(word)>1]
  return tokens

In [6]:
clean=clean_doc(text)
len(clean_doc(text))

261

In [7]:
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
  # load the doc
  doc = load_doc(filename)
  # clean_doc
  tokens = clean_doc(doc)
  # filter by vocab
  tokens= [w for w in tokens if w in vocab]
  return ' ' .join(tokens)

In [8]:
vocab=open('vocab.txt')
vocab=vocab.read().split()

In [9]:
doc_to_line('review_polarity/txt_sentoken/pos/cv026_29325.txt',vocab)

'us werent yet born rock rolled around monterey pop affectionate glimpse music influenced parents hippies otis redding jimi hendrix janis joplin mamas papas jefferson airplane documentary jampacked contagious energy give fair warning reveal ending rest film justice deserves shot outdoor concert film defies stereotype general population time sure painted faces smoke joints war room moon broadway surprisingly chooses show broad spectrum audience matter watching comes back talented musicians stir soul excitement starts music even begins young girl cleaning thousands seats asked interviewer replies feels lucky moments organized craziness john phillips leader mamas papas one concert tries get touch one band tuning member remarks finally decent sound system tell watching first moments show isnt vanity playing music love appreciation street interaction audience performer continues throughout film becomes infectious audience impossible tear eyes away janis joplin belts ballad love ball chain l

In [10]:
# load all docs in a directory
def process_train(directory, vocab):
  documents = list()
  for filename in listdir(directory):
    if not filename.startswith('cv9'):
      path = directory + '/' + filename
      doc = load_doc(path)
      tokens = clean_doc(doc, vocab)
      documents.append(tokens)
    return documents

def process_test(directory, vocab):
  documents = list()
  for filename in listdir(directory):
    if not filename.startswith('cv9'):
      path = directory + '/' + filename
      doc = load_doc(path)
      tokens = clean_doc(doc, vocab)
      documents.append(tokens)
    return documents   

In [11]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
  documents=list()
  # walk through all files in the folder
  for filename in listdir(directory):
    # skip any reviews in the test set
    if is_train and filename.startswith('cv9'):
      continue
    if not is_train and not filename.startswith('cv9'):
      continue
    # create the full path of the file to open
    path = directory + '/' + filename 
    # load the doc
    doc = load_doc(path)  
    # clean doc
    tokens = clean_doc(doc)
    # add to list
    documents.append(tokens)
  return documents 

In [12]:
lines=process_docs('review_polarity/txt_sentoken/pos',vocab,False) #(True - Test Data set)
len(lines)

100

In [13]:
# load and clean a dataset
def load_clean_dataset(vocab, is_train):
  # load documents
  neg =  process_docs('review_polarity/txt_sentoken/neg',vocab,is_train)
  pos =  process_docs('review_polarity/txt_sentoken/pos',vocab,is_train)
  docs = neg + pos
  # prepare labels
  labels = [0 for _ in range(len(neg))]+[1 for _ in range(len(pos))]
  return docs, labels

In [14]:
train, train_labels = load_clean_dataset(vocab,True)
test, test_labels = load_clean_dataset(vocab,False)

In [15]:
len(train), len(train_labels)

(1812, 1812)

In [16]:
len(test), len(test_labels)

(200, 200)

In [17]:
# define the model
def define_model(n_words):
  # define network
  model = Sequential()
  model.add(Dense(50, input_shape=(n_words,),activation= 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))

  #compile network
  model.compile(loss='binary_crossentropy' , optimizer = 'adam' , metrics=['accuracy'])
  #sumarize defined model
  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)
  return model

In [18]:
define_model(100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 5101 (19.93 KB)
Trainable params: 5101 (19.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [19]:
# fit a tokenizer

def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [20]:
# create the tokenizer
tokenizer = create_tokenizer(train)

In [21]:
# encode data
x_train = tokenizer.texts_to_matrix(train, mode='binary')

In [22]:
x_train.shape

(1812, 44277)

In [23]:
x_test = tokenizer.texts_to_matrix(test, mode= 'binary')

In [24]:
x_test.shape

(200, 44277)

In [25]:
tokenizer.word_docs

defaultdict(int,
            {'reviewed': 21,
             'one': 1569,
             'tampered': 2,
             'reviews': 65,
             'anything': 456,
             'spots': 39,
             'gonna': 43,
             'say': 527,
             'star': 358,
             'really': 783,
             'clean': 50,
             'waste': 116,
             'childish': 16,
             'sadly': 76,
             'geocities': 5,
             'given': 366,
             'yeah': 68,
             'making': 418,
             'ok': 69,
             'would': 1001,
             'disappointment': 52,
             'minute': 96,
             'imcomprehinsable': 2,
             'film': 1564,
             'make': 905,
             'laughed': 46,
             'total': 75,
             'easily': 162,
             'big': 551,
             'time': 1108,
             'kicked': 19,
             'wcw': 6,
             'quite': 422,
             'loser': 32,
             'script': 490,
             'herring': 7,


In [26]:
# define network
n_words = x_train.shape[1]
model = define_model(n_words)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 50)                2213900   
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2213951 (8.45 MB)
Trainable params: 2213951 (8.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [27]:
!pip install pydot

In [28]:
!pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 806.4 kB/s eta 0:00:00MB/s eta 0:00:01


In [29]:
model.fit(x_train,np.array(train_labels),epochs=10, batch_size=5)

2023-12-21 23:49:51.791469: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 320919696 exceeds 10% of free system memory.


Epoch 1/10
363/363 [==============================] - 14s 36ms/step - loss: 0.4263 - accuracy: 0.8046
Epoch 2/10
363/363 [==============================] - 13s 35ms/step - loss: 0.0219 - accuracy: 0.9956
Epoch 3/10
363/363 [==============================] - 13s 35ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 4/10
363/363 [==============================] - 13s 35ms/step - loss: 9.2548e-04 - accuracy: 1.0000
Epoch 5/10
363/363 [==============================] - 12s 33ms/step - loss: 4.6922e-04 - accuracy: 1.0000
Epoch 6/10
363/363 [==============================] - 12s 34ms/step - loss: 2.7235e-04 - accuracy: 1.0000
Epoch 7/10
363/363 [==============================] - 12s 32ms/step - loss: 1.7274e-04 - accuracy: 1.0000
Epoch 8/10
363/363 [==============================] - 12s 32ms/step - loss: 1.1424e-04 - accuracy: 1.0000
Epoch 9/10
363/363 [==============================] - 11s 31ms/step - loss: 7.7783e-05 - accuracy: 1.0000
Epoch 10/10
363/363 [==============================] - 12s

In [30]:
plot_model(model,show_dtype=True, show_layer_activations=True,show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [31]:
model.evaluate(x_test,np.array(test_labels),batch_size=1)

200/200 [==============================] - 1s 3ms/step - loss: 0.2436 - accuracy: 0.9200


[0.24361996352672577, 0.9200000166893005]

In [32]:
text1 = 'Best movie ever! It was great, I will definitely recommend it.'
text2 = ' This is a bad movie.'

In [33]:
# classify a review a negative or positive
def predict_sentiment(review):
    # clean
    tokens = clean_doc(review)
    #convert to line
    line=' ' .join(tokens)
    # encode
    encoded = tokenizer.texts_to_matrix([line],mode= 'binary')
    # predict statement
    yhat = model.predict(encoded, verbose=0)
    # retrieve predicted percentage and label
    percent_pos = yhat[0,0]
    if round(percent_pos) == 0:
        return (1-percent_pos), 'NEGATIVE'
    return percent_pos, 'POSITIVE'

In [34]:
percent, sentiment = predict_sentiment(text1)
print(' Review: [%s]\nSentiment: %s (%.3f%%) ' % (text1, sentiment,percent))

 Review: [Best movie ever! It was great, I will definitely recommend it.]
Sentiment: POSITIVE (0.666%) 


In [35]:
percent, sentiment = predict_sentiment(text2)
print(' Review: [%s]\nSentiment: %s (%.3f%%) ' % (text2, sentiment,percent))

 Review: [ This is a bad movie.]
Sentiment: NEGATIVE (0.692%) 
